El objetivo de este code es hacer ETL de los datos de performance de granja experimental. Los files de performance estan en formato wide y deben ser transformados a formato long.

Hay dos variaciones del ETL, cuando hay muestreo en diferentes Ages o cuando se hace muestreo solo en un Age en especifico.

Este code tiene un ejemplos de ambos ETL

La estructura de los files de performance puede variar por experimento, es por esto que cada file debe ser analizado independientemente e identificar:

1. Variables de performance de interes
2. Tomar las variables de performance que coincidan con el Sampling Date (swine) o Sampling Age (broiler) de muestreos de siwa
3. hacer ETL a esas columnas target

El ETL se divide en dos pasos:
1. Pasar de wide a long format en files de performance (Guardar performance long format)
2. Hacer merge de longformat con metadatos (Guardar metadatos + performance data)

Mas info sobre ETL ---> https://www.notion.so/ETL-experimental-farms-42a01b6be9cc48f5b5e71a1eef9f2a0a


In [2]:
import os 
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.engine.url import URL
import numpy as np
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv
import boto3
from scipy.stats import pearsonr
from scipy.stats import gmean
import statsmodels.api as sm

In [3]:
import sys
sys.path.append("/Users/sebastianbedoyamazo/Documents/siwa_git/iluma-kb")
import utils_schema as us
from utils_schema import get_otu_and_taxo
from utils_schema import get_mapfile

In [4]:
us

<module 'utils_schema' from '/Users/sebastianbedoyamazo/Documents/siwa_git/iluma-kb/utils_schema.py'>

In [5]:
load_dotenv("/Users/sebastianbedoyamazo/Documents/Data-exploration-and-others/.env", override=True)

True

In [6]:
os.environ
sys.path.append("/Users/sebastianbedoyamazo/Documents/Data-exploration-and-others")
import utils_dataexp as u
u

Reading aws keys from .env successfully!!


<module 'utils_dataexp' from '/Users/sebastianbedoyamazo/Documents/Data-exploration-and-others/utils_dataexp.py'>

In [7]:
s3_client = boto3.client("s3")
s3_resource = boto3.resource("s3")

In [8]:
url = URL.create(
    drivername='redshift+redshift_connector',
    host='iluma-kb-1.cn4ff1ztoyt9.us-east-1.redshift.amazonaws.com',
    port=5439,
    database='dev',
    username='awsuser',
    password='Ilumasiwa1'
)

engine = create_engine(url)
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

redshift_metadata = sa.MetaData(bind=session.bind)

# Example: different Sampling Ages

# E294

In [11]:
E294 = pd.read_excel("/Users/sebastianbedoyamazo/Documents/siwa_git/swine_data/performance_data/E294 Data Base FINAL.xlsx", sheet_name='SAS Fixed', header=0, nrows=45)
E294.head()

,Obs,Rep,Corral,Trt,Ganoderma,Antibiotics$,Sexo$,StDate,StWt,W1Date,...,W7FCR,Phase3ADG,Phase3ADFI,Phase3FCR,StEndADG,StEndADFI,StEndFCR,StEndFeedPrice,StEndFeedCost,FeedCostPerkgGain
0,1,1,1,2,0,No,M,43424,5.280000,43431,...,1.784157,694.920635,1088.042328,1.565707,580.524781,846.705539,1.458517,NaN,NaN,NaN
1,2,1,2,3,200,No,M,43424,5.291429,43431,...,1.737737,706.349206,1096.719577,1.552659,550.087464,816.384840,1.484100,NaN,NaN,NaN
2,3,1,3,5,400,Yes,M,43424,5.288571,43431,...,1.689907,717.248677,1019.470899,1.421363,565.947522,774.110787,1.367814,NaN,NaN,NaN
3,4,1,4,1,0,Yes,M,43424,5.288571,43431,...,1.889883,706.031746,1196.402116,1.694544,596.559767,918.425656,1.539537,NaN,NaN,NaN
4,5,1,5,4,400,No,M,43424,5.280000,43431,...,1.645614,667.936508,1049.100529,1.570659,527.405248,786.239067,1.490768,NaN,NaN,NaN


In [12]:
E294.columns

Index(['Obs', 'Rep', 'Corral', 'Trt', 'Ganoderma', 'Antibiotics$', 'Sexo$',
       'StDate', 'StWt', 'W1Date', 'W1Wt', 'W1WtCV', 'W1ADG', 'W1ADFI',
       'W1FCR', 'W2Date', 'W2Wt', 'W2ADG', 'W2ADFI', 'W2FCR', 'W3Date', 'W3Wt',
       'W3WtCV', 'W3ADG', 'W3ADFI', 'W3FCR', 'Phase2ADG', 'Phase2ADFI',
       'Phase2FCR', 'W4Date', 'W4Wt', 'W4ADG', 'W4ADFI', 'W4FCR', 'W5Date',
       'W5Wt', 'W5ADG', 'W5ADFI', 'W5FCR', 'W6Date', 'W6Wt', 'W6ADG', 'W6ADFI',
       'W6FCR', 'W7Date', 'W7Wt', 'W7WtCV', 'W7ADG', 'W7ADFI', 'W7FCR',
       'Phase3ADG', 'Phase3ADFI', 'Phase3FCR', 'StEndADG', 'StEndADFI',
       'StEndFCR', 'StEndFeedPrice', 'StEndFeedCost', 'FeedCostPerkgGain'],
      dtype='object')

In [483]:
dfE294 = E294[['Corral', 'Trt', 'Sexo$', 'StWt', 'W3Wt', 'W3ADG', 'W3ADFI', 'W3FCR',
             'W5Wt', 'W5ADG', 'W5ADFI', 'W5FCR']]
dfE294.head()

,Corral,Trt,Sexo$,StWt,W3Wt,W3ADG,W3ADFI,W3FCR,W5Wt,W5ADG,W5ADFI,W5FCR
0,1,2,M,5.280000,14.962857,586.785714,777.500000,1.325015,24.408571,698.928571,1063.571429,1.521717
1,2,3,M,5.291429,13.174286,453.571429,645.714286,1.423622,22.351429,689.285714,1042.500000,1.512435
2,3,5,M,5.288571,13.654286,515.357143,691.428571,1.341649,23.308571,746.071429,1062.857143,1.424605
3,4,1,M,5.288571,15.457143,529.642857,763.928571,1.442347,24.711429,707.142857,1095.714286,1.549495
4,5,4,M,5.280000,13.088571,438.214286,634.285714,1.447433,21.451429,637.500000,1032.142857,1.619048


In [484]:
dfE294['Corral'] = dfE294['Corral'].astype(float).astype(int)
dfE294['Trt'] = dfE294['Trt'].astype(float).astype(int)

/var/folders/8_/97xnhkcj3qvf45tl2cgtc7vc0000gp/T/ipykernel_5328/2467561216.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfE294['Corral'] = dfE294['Corral'].astype(float).astype(int)
/var/folders/8_/97xnhkcj3qvf45tl2cgtc7vc0000gp/T/ipykernel_5328/2467561216.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfE294['Trt'] = dfE294['Trt'].astype(float).astype(int)


In [534]:
dfE294 = dfE294.rename(columns={'Corral':'Pen', 'Sexo$': 'Sex', 
                       'StWt': 'BW23',
                       'W3Wt':'BW48', 'W3ADG':'ADG23_48', 'W3ADFI':'ADFI23_48', 'W3FCR':'FCR23_48',
                       'W5Wt':'BW63', 'W5ADG':'ADG48_63', 'W5ADFI':'ADFI48_63', 'W5FCR':'FCR48_63'})

In [535]:
dfE294['ProjectID'] = "E294"  
dfE294.head()

,Pen,Trt,Sex,BW23,BW48,ADG23_48,ADFI23_48,FCR23_48,BW63,ADG48_63,ADFI48_63,FCR48_63,ProjectID
0,1,2,M,5.280000,14.962857,586.785714,777.500000,1.325015,24.408571,698.928571,1063.571429,1.521717,E294
1,2,3,M,5.291429,13.174286,453.571429,645.714286,1.423622,22.351429,689.285714,1042.500000,1.512435,E294
2,3,5,M,5.288571,13.654286,515.357143,691.428571,1.341649,23.308571,746.071429,1062.857143,1.424605,E294
3,4,1,M,5.288571,15.457143,529.642857,763.928571,1.442347,24.711429,707.142857,1095.714286,1.549495,E294
4,5,4,M,5.280000,13.088571,438.214286,634.285714,1.447433,21.451429,637.500000,1032.142857,1.619048,E294


In [536]:
dfE294.columns

Index(['Pen', 'Trt', 'Sex', 'BW23', 'BW48', 'ADG23_48', 'ADFI23_48',
       'FCR23_48', 'BW63', 'ADG48_63', 'ADFI48_63', 'FCR48_63', 'ProjectID'],
      dtype='object')

In [537]:
df_long1 = pd.concat([dfE294.melt(id_vars=['ProjectID', 'Pen', 'Trt', 'Sex'], value_vars=['BW23', 'BW48', 'BW63'], var_name='Age', value_name='BW')
])
df_long1['Age'] = df_long1['Age'].str[-2:]
df_long1.head()

,ProjectID,Pen,Trt,Sex,Age,BW
0,E294,1,2,M,23,5.280000
1,E294,2,3,M,23,5.291429
2,E294,3,5,M,23,5.288571
3,E294,4,1,M,23,5.288571
4,E294,5,4,M,23,5.280000


In [538]:
df_long2 = pd.concat([dfE294.melt(id_vars=['ProjectID', 'Pen', 'Trt', 'Sex'], value_vars=['ADG23_48', 'ADG48_63'], var_name='Age', value_name='ADGbefore')
])
df_long2['Age'] = df_long2['Age'].str[-2:]
df_long2.head()

,ProjectID,Pen,Trt,Sex,Age,ADGbefore
0,E294,1,2,M,48,586.785714
1,E294,2,3,M,48,453.571429
2,E294,3,5,M,48,515.357143
3,E294,4,1,M,48,529.642857
4,E294,5,4,M,48,438.214286


In [539]:
df_long3 = pd.concat([dfE294.melt(id_vars=['ProjectID', 'Pen', 'Trt', 'Sex'], value_vars=['ADFI23_48', 'ADFI48_63'], var_name='Age', value_name='ADFIbefore')
])
df_long3['Age'] = df_long3['Age'].str[-2:]
df_long3.head()

,ProjectID,Pen,Trt,Sex,Age,ADFIbefore
0,E294,1,2,M,48,777.500000
1,E294,2,3,M,48,645.714286
2,E294,3,5,M,48,691.428571
3,E294,4,1,M,48,763.928571
4,E294,5,4,M,48,634.285714


In [540]:
df_long4 = pd.concat([dfE294.melt(id_vars=['ProjectID', 'Pen', 'Trt', 'Sex'], value_vars=['FCR23_48', 'FCR48_63'], var_name='Age', value_name='FCRbefore')
])
df_long4['Age'] = df_long4['Age'].str[-2:]
df_long4.head()

,ProjectID,Pen,Trt,Sex,Age,FCRbefore
0,E294,1,2,M,48,1.325015
1,E294,2,3,M,48,1.423622
2,E294,3,5,M,48,1.341649
3,E294,4,1,M,48,1.442347
4,E294,5,4,M,48,1.447433


In [541]:
E294_long = df_long1.merge(df_long2, on=['ProjectID', 'Pen', 'Trt', 'Sex', 'Age'], how='left') \
               .merge(df_long3, on=['ProjectID', 'Pen', 'Trt', 'Sex', 'Age'], how='left') \
               .merge(df_long4, on=['ProjectID', 'Pen', 'Trt', 'Sex', 'Age'], how='left')
E294_long.head()


,ProjectID,Pen,Trt,Sex,Age,BW,ADGbefore,ADFIbefore,FCRbefore
0,E294,1,2,M,23,5.280000,NaN,NaN,NaN
1,E294,2,3,M,23,5.291429,NaN,NaN,NaN
2,E294,3,5,M,23,5.288571,NaN,NaN,NaN
3,E294,4,1,M,23,5.288571,NaN,NaN,NaN
4,E294,5,4,M,23,5.280000,NaN,NaN,NaN


# metadata

llamar metadata del experimento al cual se le hara el ETL, es importante que el file de metadatos contenga las siguientes columnas:
1. House y Pen
2. SampleID

In [ ]:
query = "SELECT * FROM microbiome as M LEFT JOIN kit as K on M.kitid = K.kitid LEFT JOIN animal as A on M.animalid = A.animalid WHERE projectid = 'XXXX';"
metadata_E294 = pd.read_sql(query, engine)
print(metadata_E294.shape)
print(metadata_E294.columns)
#metadata.groupby(['treatment', 'treatmentnumber']).count()

In [322]:
metadata.head()

,sampleid,fullsampleid,kitid,runid,animalid,samplelocation,alphashannon,alphaobserved,kitid,projectid,age,treatment,treatmentnumber,client,animalid,animaltype,animalnumber,house,pen
0,H058_02F,H058_02F-M,H058,20190404,H058_02,feces,1.965785,5.0,H058,PilotStudy,0.0,La Rivera,13,Acondesa,H058_02,Swines,02,None,None
1,H058_06F,H058_06F-M,H058,20190404,H058_06,feces,2.500995,9.0,H058,PilotStudy,0.0,La Rivera,13,Acondesa,H058_06,Swines,06,None,None
2,H058_07F,H058_07F-M,H058,20190404,H058_07,feces,2.264098,9.0,H058,PilotStudy,0.0,La Rivera,13,Acondesa,H058_07,Swines,07,None,None
3,H059_01F,H059_01F-M,H059,20190404,H059_01,feces,1.840196,6.0,H059,PilotStudy,0.0,Villa Sonia,14,Acondesa,H059_01,Swines,01,None,None
4,H059_10F,H059_10F-M,H059,20190404,H059_10,feces,2.276140,14.0,H059,PilotStudy,0.0,Villa Sonia,14,Acondesa,H059_10,Swines,10,None,None


# Pen/House Files

Para el ETL actual utilice los archivos de House y Pen descargados directamente de S3, cuando la base de datos este actualizada no sera necesario

In [8]:
penE294 = pd.read_csv('/Users/sebastianbedoyamazo/Documents/siwa_git/swine_data/house_pen/E294_pen.tsv', delimiter='\t')
penE319 = pd.read_csv('/Users/sebastianbedoyamazo/Documents/siwa_git/swine_data/house_pen/E319_pen.tsv', delimiter='\t')

penE294 = penE294.rename(columns={'SampleID':'fullsampleid'})
penE319 = penE319.rename(columns={'SampleID':'fullsampleid'})


In [9]:
penE294.head(3)

,SampleID,Pen,House
0,H031_01F-M,1,1
1,H031_02F-M,1,1
2,H030_03F-M,10,1


In [543]:
metadata_E294['projectid'] = metadata_E294['projectid'].str.strip()
metadata_E294.head(2)

/var/folders/8_/97xnhkcj3qvf45tl2cgtc7vc0000gp/T/ipykernel_5328/3923513575.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata_E294['projectid'] = metadata_E294['projectid'].str.strip()


,sampleid,fullsampleid,kitid,runid,animalid,samplelocation,alphashannon,alphaobserved,kitid,projectid,age,treatment,treatmentnumber,client,animalid,animaltype,animalnumber,house,pen
295,H030_03F,H030_03F-M,H030,20190728,H030_03,feces,6.518976,243.0,H030,E294,23.0,PC = Positive control (with antibiotic growth ...,1,Iluma,H030_03,Swines,03,None,None
296,H030_05F,H030_05F-M,H030,20190728,H030_05,feces,6.631080,237.0,H030,E294,23.0,PC = Positive control (with antibiotic growth ...,1,Iluma,H030_05,Swines,05,None,None


In [544]:
metadata_E294 = metadata_E294.loc[:, ~metadata_E294.columns.isin(['pen', 'house'])]
metadata_E294.head(2)

,sampleid,fullsampleid,kitid,runid,animalid,samplelocation,alphashannon,alphaobserved,kitid,projectid,age,treatment,treatmentnumber,client,animalid,animaltype,animalnumber
295,H030_03F,H030_03F-M,H030,20190728,H030_03,feces,6.518976,243.0,H030,E294,23.0,PC = Positive control (with antibiotic growth ...,1,Iluma,H030_03,Swines,03
296,H030_05F,H030_05F-M,H030,20190728,H030_05,feces,6.631080,237.0,H030,E294,23.0,PC = Positive control (with antibiotic growth ...,1,Iluma,H030_05,Swines,05


In [545]:
metadata_E294_pen = metadata_E294.merge(penE294, left_on='fullsampleid', right_on='SampleID', how='inner')
metadata_E294_pen.head(2)

,sampleid,fullsampleid,kitid,runid,animalid,samplelocation,alphashannon,alphaobserved,kitid,projectid,age,treatment,treatmentnumber,client,animalid,animaltype,animalnumber,SampleID,Pen,House
0,H030_03F,H030_03F-M,H030,20190728,H030_03,feces,6.518976,243.0,H030,E294,23.0,PC = Positive control (with antibiotic growth ...,1,Iluma,H030_03,Swines,03,H030_03F-M,10,1
1,H030_05F,H030_05F-M,H030,20190728,H030_05,feces,6.631080,237.0,H030,E294,23.0,PC = Positive control (with antibiotic growth ...,1,Iluma,H030_05,Swines,05,H030_05F-M,13,1


In [546]:
metadata_E294_pen = metadata_E294_pen.rename(columns={'projectid':'ProjectID', 'age':'Age', 'treatmentnumber':'Trt'})

In [549]:
metadata_E294_pen['Pen'].unique()

array([10, 13, 20, 28, 37, 40,  1, 21, 29, 35, 42, 48,  7, 15, 19,  2, 25,
       31, 41, 44,  6, 14, 17, 27, 43,  5, 11, 16, 22,  3, 30, 34, 39, 47,
        8, 32,  4, 36, 46,  9, 45, 12, 18, 38, 33])

In [550]:
E294_long['Pen'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
       38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48])

In [551]:
metadata_E294_pen['Trt'].unique()

array([1, 2, 3, 4, 5])

In [552]:
E294_long['Trt'].unique()

array([2, 3, 5, 1, 4])

In [553]:
metadata_E294_pen['Age'].unique()

array(['23.0', '48.0', '63.0'], dtype=object)

In [554]:
E294_long['Age'].unique()

array(['23', '48', '63'], dtype=object)

In [555]:
metadata_E294_pen['Age'] = metadata_E294_pen['Age'].str.replace('.0', '')
metadata_E294_pen['Age'].unique()

/var/folders/8_/97xnhkcj3qvf45tl2cgtc7vc0000gp/T/ipykernel_5328/814742737.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  metadata_E294_pen['Age'] = metadata_E294_pen['Age'].str.replace('.0', '')


array(['23', '48', '63'], dtype=object)

In [556]:
metadata_E294_pen['ProjectID'].unique()

array(['E294'], dtype=object)

In [557]:
E294_long[['ProjectID', 'Age', 'Trt', 'Pen']] = E294_long[['ProjectID', 'Age', 'Trt', 'Pen']].astype(object)
metadata_E294_pen[['ProjectID', 'Age', 'Trt', 'Pen']] = metadata_E294_pen[['ProjectID', 'Age', 'Trt', 'Pen']].astype(object)

In [558]:
E294_long.dtypes

ProjectID      object
Pen            object
Trt            object
Sex            object
Age            object
BW            float64
ADGbefore     float64
ADFIbefore    float64
FCRbefore     float64
dtype: object

In [559]:
metadata_E294_pen.dtypes

sampleid           object
fullsampleid       object
kitid              object
runid              object
animalid           object
samplelocation     object
alphashannon      float64
alphaobserved     float64
kitid              object
ProjectID          object
Age                object
treatment          object
Trt                object
client             object
animalid           object
animaltype         object
animalnumber       object
SampleID           object
Pen                object
House               int64
dtype: object

In [560]:
metadata_E294_pen = metadata_E294_pen.drop(metadata_E294_pen.columns[[2, 14, 17]], axis=1)
print(metadata_E294_pen.columns)


Index(['sampleid', 'fullsampleid', 'runid', 'samplelocation', 'alphashannon',
       'alphaobserved', 'ProjectID', 'Age', 'treatment', 'Trt', 'client',
       'animaltype', 'animalnumber', 'Pen', 'House'],
      dtype='object')


In [576]:
metadata_E294_pen_perf = pd.merge(metadata_E294_pen, E294_long, on =['ProjectID', 'Age', 'Trt', 'Pen'], how='left')
print(metadata_E294_pen_perf.shape)
print(metadata_E294_pen_perf.columns)
metadata_E294_pen_perf.head(2)

(183, 20)
Index(['sampleid', 'fullsampleid', 'runid', 'samplelocation', 'alphashannon',
       'alphaobserved', 'ProjectID', 'Age', 'treatment', 'Trt', 'client',
       'animaltype', 'animalnumber', 'Pen', 'House', 'Sex', 'BW', 'ADGbefore',
       'ADFIbefore', 'FCRbefore'],
      dtype='object')


,sampleid,fullsampleid,runid,samplelocation,alphashannon,alphaobserved,ProjectID,Age,treatment,Trt,client,animaltype,animalnumber,Pen,House,Sex,BW,ADGbefore,ADFIbefore,FCRbefore
0,H030_03F,H030_03F-M,20190728,feces,6.518976,243.0,E294,23,PC = Positive control (with antibiotic growth ...,1,Iluma,Swines,03,10,1,M,5.328571,NaN,NaN,NaN
1,H030_05F,H030_05F-M,20190728,feces,6.631080,237.0,E294,23,PC = Positive control (with antibiotic growth ...,1,Iluma,Swines,05,13,1,M,5.411429,NaN,NaN,NaN


In [705]:
metadata_E294_pen_perf.dtypes

sampleid           object
fullsampleid       object
runid              object
samplelocation     object
alphashannon      float64
alphaobserved     float64
ProjectID          object
Age                object
treatment          object
Trt                object
client             object
animaltype         object
animalnumber       object
Pen                object
House               int64
Sex                object
BW                float64
ADGbefore         float64
ADFIbefore        float64
FCRbefore         float64
dtype: object

In [630]:
E294_long.to_csv("/Users/sebastianbedoyamazo/Documents/siwa_git/swine_data/para_merge/E294_long.csv", header=True, index=False)
metadata_E294_pen_perf.to_csv("/Users/sebastianbedoyamazo/Documents/siwa_git/swine_data/para_merge/metadata_E294_pen_perf.csv", header=True, index=False)

# Example: Just One Sampling Age

# E319

In [203]:
E319 = pd.read_excel("/Users/sebastianbedoyamazo/Documents/siwa_git/swine_data/performance_data/E-319 Data Base Final.xlsx", sheet_name='SAS', header=0, nrows=69)
E319.head()

,Obs,Rep,Pen,Trt,Sex$,P0Date,P0Count,P0Wt,P1Date,P1Count,...,Phase3ADG,Phase3ADFI,Phase3FCR,Phase3Days,StoFADG,StoFADFI,StoFFCR,TotalDays,MortCount,Mortality
0,1,1,2,4,M,2020-05-29,21,5.619048,2020-06-05,21,...,0.482993,0.748299,1.549296,21,0.376093,0.539359,1.434109,49,0,0.000000
1,2,1,3,1,M,2020-05-29,21,5.666667,2020-06-05,21,...,0.576190,0.854762,1.483471,21,0.416148,0.577533,1.387804,49,1,4.761905
2,3,1,4,6,M,2020-05-29,21,5.809524,2020-06-05,21,...,0.519048,0.850000,1.637615,21,0.387827,0.578471,1.491569,49,1,4.761905
3,4,1,5,2,M,2020-05-29,21,5.833333,2020-06-05,21,...,0.540816,0.786848,1.454927,21,0.414480,0.561419,1.354513,49,0,0.000000
4,5,1,6,5,M,2020-05-29,21,5.857143,2020-06-05,21,...,0.526077,0.852608,1.620690,21,0.393586,0.597668,1.518519,49,0,0.000000


In [711]:
E319.columns

Index(['Obs', 'Rep', 'Pen', 'Trt', 'Sex$', 'P0Date', 'P0Count', 'P0Wt',
       'P1Date', 'P1Count', 'P1Wt', 'P1ADG', 'P1ADFI', 'P1FCR', 'P1Days',
       'Phase1ADG', 'Phase1ADFI', 'Phase1FCR', 'Phase1Days', 'P2Date',
       'P2Count', 'P2Wt', 'P2ADG', 'P2ADFI', 'P2FCR', 'P2Days', 'P3Date',
       'P3Count', 'P3Wt', 'P3ADG', 'P3ADFI', 'P3FCR', 'P3Days', 'P4Date',
       'P4Count', 'P4Wt', 'P4ADG', 'P4ADFI', 'P4FCR', 'P4Days', 'Phase2ADG',
       'Phase2ADFI', 'Phase2FCR', 'Phase2Days', 'P5Date', 'P5Count', 'P5Wt',
       'P5ADG', 'P5ADFI', 'P5FCR', 'P5Days', 'P6Date', 'P6Count', 'P6Wt',
       'P6ADG', 'P6ADFI', 'P6FCR', 'P6Days', 'P7Date', 'P7Count', 'P7Wt',
       'P7ADG', 'P7ADFI', 'P7FCR', 'P7Days', 'Phase3ADG', 'Phase3ADFI',
       'Phase3FCR', 'Phase3Days', 'StoFADG', 'StoFADFI', 'StoFFCR',
       'TotalDays', 'MortCount', 'Mortality'],
      dtype='object')

In [712]:
dfE319 = E319[['Pen', 'Trt', 'Sex$', 
               'P3Wt', 'P3ADG', 'P3ADFI', 'P3FCR']]
dfE319.head()

,Pen,Trt,Sex$,P3Wt,P3ADG,P3ADFI,P3FCR
0,2,4,M,11.047619,0.316327,0.353741,1.118280
1,3,1,M,11.100000,0.296503,0.384615,1.297170
2,4,6,M,10.800000,0.283929,0.307143,1.081761
3,5,2,M,11.285714,0.312925,0.374150,1.195652
4,6,5,M,10.619048,0.275510,0.340136,1.234568


In [713]:
dfE319['Pen'] = dfE319['Pen'].astype(float).astype(int)
dfE319['Trt'] = dfE319['Trt'].astype(float).astype(int)

/var/folders/8_/97xnhkcj3qvf45tl2cgtc7vc0000gp/T/ipykernel_5328/2852398196.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfE319['Pen'] = dfE319['Pen'].astype(float).astype(int)
/var/folders/8_/97xnhkcj3qvf45tl2cgtc7vc0000gp/T/ipykernel_5328/2852398196.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfE319['Trt'] = dfE319['Trt'].astype(float).astype(int)


In [714]:
dfE319 = dfE319.rename(columns={'Sex$': 'Sex',
                       'P3Wt':'BW', 'P3ADG':'ADGbefore', 'P3ADFI':'ADFIbefore', 'P3FCR':'FCRbefore'})
dfE319['ProjectID'] = "E319" 
dfE319['Age'] = "42" 
dfE319.head()

,Pen,Trt,Sex,BW,ADGbefore,ADFIbefore,FCRbefore,ProjectID,Age
0,2,4,M,11.047619,0.316327,0.353741,1.118280,E319,42
1,3,1,M,11.100000,0.296503,0.384615,1.297170,E319,42
2,4,6,M,10.800000,0.283929,0.307143,1.081761,E319,42
3,5,2,M,11.285714,0.312925,0.374150,1.195652,E319,42
4,6,5,M,10.619048,0.275510,0.340136,1.234568,E319,42


In [715]:
E319_long = dfE319

In [716]:
metadata_E319['projectid'] = metadata_E319['projectid'].str.strip()
metadata_E319.head(2)

/var/folders/8_/97xnhkcj3qvf45tl2cgtc7vc0000gp/T/ipykernel_5328/3840833397.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata_E319['projectid'] = metadata_E319['projectid'].str.strip()


,sampleid,fullsampleid,kitid,runid,animalid,samplelocation,alphashannon,alphaobserved,kitid,projectid,age,treatment,treatmentnumber,client,animalid,animaltype,animalnumber,house,pen
193,0028_01F,0028_01F-M,0028,20210527,0028_01,feces,4.876227,187.0,0028,E319,42,1,1,Iluma,0028_01,Swines,01,None,None
194,0028_04F,0028_04F-M,0028,20210527,0028_04,feces,5.011337,193.0,0028,E319,42,1,1,Iluma,0028_04,Swines,04,None,None


In [717]:
metadata_E319 = metadata_E319.loc[:, ~metadata_E319.columns.isin(['pen', 'house'])]
metadata_E319.head(2)

,sampleid,fullsampleid,kitid,runid,animalid,samplelocation,alphashannon,alphaobserved,kitid,projectid,age,treatment,treatmentnumber,client,animalid,animaltype,animalnumber
193,0028_01F,0028_01F-M,0028,20210527,0028_01,feces,4.876227,187.0,0028,E319,42,1,1,Iluma,0028_01,Swines,01
194,0028_04F,0028_04F-M,0028,20210527,0028_04,feces,5.011337,193.0,0028,E319,42,1,1,Iluma,0028_04,Swines,04


In [718]:
metadata_E319_pen = metadata_E319.merge(penE319, on=['fullsampleid'], how='inner')
metadata_E319_pen.head(2)

,sampleid,fullsampleid,kitid,runid,animalid,samplelocation,alphashannon,alphaobserved,kitid,projectid,age,treatment,treatmentnumber,client,animalid,animaltype,animalnumber,Pen,House
0,0028_01F,0028_01F-M,0028,20210527,0028_01,feces,4.876227,187.0,0028,E319,42,1,1,Iluma,0028_01,Swines,01,3,1
1,0028_04F,0028_04F-M,0028,20210527,0028_04,feces,5.011337,193.0,0028,E319,42,1,1,Iluma,0028_04,Swines,04,31,1


In [719]:
metadata_E319_pen = metadata_E319_pen.rename(columns={'projectid':'ProjectID', 'age':'Age', 'treatmentnumber':'Trt'})


In [720]:
print(metadata_E319_pen['Pen'].unique())
print(E319_long['Pen'].unique())

[ 3 31 47 56 65 11 39 54 62 12 21 38 70  8 16 28 35 40 46 55 13 20 42 66
  7 15 23 29 44 71  2 19 26 33 49 57 60  4 63 69]
[ 2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49
 52 53 54 55 56 57 60 61 62 63 64 65 66 67 68 69 70 71]


In [721]:
print(metadata_E319_pen['Trt'].unique())
print(E319_long['Trt'].unique())

[1 3 4 6]
[4 1 6 2 5 3]


In [722]:
E319_long[['ProjectID', 'Age', 'Trt', 'Pen']] = E319_long[['ProjectID', 'Age', 'Trt', 'Pen']].astype(object)
metadata_E319_pen[['ProjectID', 'Age', 'Trt', 'Pen']] = metadata_E319_pen[['ProjectID', 'Age', 'Trt', 'Pen']].astype(object)


In [723]:
E319_long.dtypes


Pen            object
Trt            object
Sex            object
BW            float64
ADGbefore     float64
ADFIbefore    float64
FCRbefore     float64
ProjectID      object
Age            object
dtype: object

In [724]:
metadata_E319_pen.dtypes


sampleid           object
fullsampleid       object
kitid              object
runid              object
animalid           object
samplelocation     object
alphashannon      float64
alphaobserved     float64
kitid              object
ProjectID          object
Age                object
treatment          object
Trt                object
client             object
animalid           object
animaltype         object
animalnumber       object
Pen                object
House               int64
dtype: object

In [725]:
metadata_E319_pen = metadata_E319_pen.drop(metadata_E319_pen.columns[[2, 14]], axis=1)
print(metadata_E319_pen.columns)

Index(['sampleid', 'fullsampleid', 'runid', 'samplelocation', 'alphashannon',
       'alphaobserved', 'ProjectID', 'Age', 'treatment', 'Trt', 'client',
       'animaltype', 'animalnumber', 'Pen', 'House'],
      dtype='object')


In [726]:
metadata_E319_pen_perf = pd.merge(metadata_E319_pen, E319_long, on =['ProjectID', 'Age', 'Trt', 'Pen'], how='left')
print(metadata_E319_pen_perf.shape)
print(metadata_E319_pen_perf.columns)
metadata_E319_pen_perf.head(2)

(40, 20)
Index(['sampleid', 'fullsampleid', 'runid', 'samplelocation', 'alphashannon',
       'alphaobserved', 'ProjectID', 'Age', 'treatment', 'Trt', 'client',
       'animaltype', 'animalnumber', 'Pen', 'House', 'Sex', 'BW', 'ADGbefore',
       'ADFIbefore', 'FCRbefore'],
      dtype='object')


,sampleid,fullsampleid,runid,samplelocation,alphashannon,alphaobserved,ProjectID,Age,treatment,Trt,client,animaltype,animalnumber,Pen,House,Sex,BW,ADGbefore,ADFIbefore,FCRbefore
0,0028_01F,0028_01F-M,20210527,feces,4.876227,187.0,E319,42,1,1,Iluma,Swines,01,3,1,M,11.100000,0.296503,0.384615,1.297170
1,0028_04F,0028_04F-M,20210527,feces,5.011337,193.0,E319,42,1,1,Iluma,Swines,04,31,1,M,11.928571,0.329932,0.510204,1.546392


In [728]:
E319_long.to_csv("/Users/sebastianbedoyamazo/Documents/siwa_git/swine_data/para_merge/E319_long.csv", header=True, index=False)
metadata_E319_pen_perf.to_csv("/Users/sebastianbedoyamazo/Documents/siwa_git/swine_data/para_merge/metadata_E319_pen_perf.csv", header=True, index=False)